In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()


False

In [4]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [5]:
#load the data from website
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
#making the chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_spliter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
final_document = text_spliter.split_documents(docs)
 

In [7]:
print(final_document)

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Get started with LangSmith | 🦜️🛠️ LangSmith'), Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started w

In [8]:
from langchain_openai import OpenAIEmbeddings
embedding=OpenAIEmbeddings()


In [9]:
import numpy
import faiss

print("NumPy version:", numpy.__version__)
print("FAISS version:", faiss.__version__)


NumPy version: 1.26.4
FAISS version: 1.7.4


In [ ]:
from langchain_community.vectorstores import FAISS
vector = FAISS.from_documents(final_document,embedding)


In [11]:
print(vector)

In [12]:
#query form a vector db
query = "Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug."
result = vector.similarity_search(query)
result[0].page_content

"LangSmith + LangChain OSSLangSmith is framework-agnostic —\xa0it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.\nFor more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.\nObservability\u200b\nObservability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.\nThis is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prototyping, to beta testing, to production."

In [13]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model ="gpt-4o")

In [15]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
promt = ChatPromptTemplate.from_template(
    """
        Answer the following question based only on the provided context:
        <context>
        {context}
        </context>
    """
)
document_chain =  create_stuff_documents_chain(llm,promt)
document_chain


RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        Answer the following question based only on the provided context:\n        <context>\n        {context}\n        </context>\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x1262211b0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x126bdafb0>, root_client=<openai.OpenAI object at 0x1269472b0>, root_async_client=<openai.AsyncOpenAI object at 0x126221480>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, c

In [20]:
from langchain_core.documents import Document
document_chain.invoke({

    "input" : "Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.",
    "context": [Document(page_content="Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.")]
}
)

'Why is observability especially important for LLM applications?\n\nObservability is especially important for LLM (Large Language Model) applications because LLMs are non-deterministic by nature, which means they can produce unexpected results. This non-determinism makes them more challenging to debug compared to traditional software applications.'

In [22]:
retriver=vector.as_retriever()
from langchain.chains import create_retrieval_chain
retriver_chain=create_retrieval_chain(retriver,document_chain)
retriver_chain


RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x122f9b1f0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        Answer the following question based only on the provided context:\n        <context>\n        {context}\n        </context>\n    '), additional_kw

In [23]:
#get the response from the llm
response = retriver_chain.invoke({"input":"Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug."})


In [25]:
response['answer']

"What is LangSmith, and what are some of its features for building LLM applications?\n\nLangSmith is a platform for building production-grade LLM (Large Language Model) applications. It is framework-agnostic, meaning it can be used with or without LangChain's open source frameworks, such as langchain and langgraph. Some of its key features include:\n\n1. **Observability**: LangSmith provides LLM-native observability, allowing for meaningful insights into application performance. This includes analyzing traces, configuring metrics, dashboards, and alerts, which are crucial due to the non-deterministic nature of LLMs.\n\n2. **Evals**: LangSmith helps in evaluating applications over production traffic, scoring application performance, and gathering human feedback. It makes the creation and execution of high-quality evaluations easy with the help of its SDK and UI.\n\n3. **Prompt Engineering**: It facilitates the iteration on prompts, providing automatic version control and collaboration f

In [26]:
response

{'input': 'Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.',
 'context': [Document(id='74bd6a60-538d-4b76-91f2-8f23a9546a30', metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content="LangSmith + LangChain OSSLangSmith is framework-agnostic —\xa0it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.\nFor more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.\nObservability\u200b\nObservability is important for any software application, but especially

In [27]:
response['context']

[Document(id='74bd6a60-538d-4b76-91f2-8f23a9546a30', metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content="LangSmith + LangChain OSSLangSmith is framework-agnostic —\xa0it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.\nFor more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.\nObservability\u200b\nObservability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.\nThis is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get m